<a href="https://colab.research.google.com/github/luiscaballeroramos/DeCoder/blob/master/DeCoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PyPDF2



In [1]:
pip install PyPDF2

     |████████████████████████████████| 81kB 2.4MB/s 
  Created wheel for PyPDF2: filename=PyPDF2-1.26.0-cp36-none-any.whl size=61086 sha256=72bbb23967ab3998cb560e57ee971001ea74a217cb1c2ef89598b5bbf8ab3267
  Stored in directory: /root/.cache/pip/wheels/53/84/19/35bc977c8bf5f0c23a8a011aa958acd4da4bbd7a229315c1b7
Successfully built PyPDF2


In [2]:
! git clone https://github.com/luiscaballeroramos/DeCoder

Cloning into 'DeCoder'...
remote: Enumerating objects: 6, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 6 (delta 0), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (6/6), done.
